In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
df = pd.read_csv("../data/full_df_cleaned_v3.csv")

In [3]:
df.sample(1)

,Diagnostic,file,target_init,Patient Age,Patient Sex,Target,tarstr,N,D,G,C,A,H,M,O,filename
5966,normal fundus,4672_left.jpg,"[0, 1, 0, 0, 0, 0, 0, 0]",65,Male,"[1, 0, 0, 0, 0, 0, 0, 0]",N,1,0,0,0,0,0,0,0,4672_left_65_Male_N.jpg


In [4]:
# tarNumber = 0 --> N
# tarNumber = 1 --> D ...

def tarNumberGenerator(characterInput):
    if(characterInput == 'N'):
        return 1
    elif(characterInput == 'D'):
        return 2
    elif(characterInput == 'G'):
        return 3
    elif(characterInput == 'C'):
        return 4
    elif(characterInput == 'A'):
        return 5
    elif(characterInput == 'H'):
        return 6
    elif(characterInput == 'M'):
        return 7
    elif(characterInput == 'O'):
        return 8
    else:
        return -1
    
df['tarNum'] = df['tarstr'].apply(tarNumberGenerator)

In [5]:
IMAGE_PATH = 'D:/College/6th_Semester/Mini_Project/data/preprocessed_images2/'

In [6]:
df['filepath'] = IMAGE_PATH + df['filename']

In [7]:
# Load the image data
import imageio.v2 as imageio
img_data = []
number_id_nofile = []

for i in range(len(df)):
  try:
    img_data.append(imageio.imread(df['filepath'][i]))
  except:
    number_id_nofile.append(df.index[i])

In [8]:
img_data[0].shape

(256, 256, 3)

In [9]:
img_data_array = np.array(img_data)

In [10]:
x = img_data_array
y = df['tarNum']

In [11]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.3, stratify=y)

In [12]:
#Normalize
X_train = X_train / 255
X_test = X_test/255

In [17]:
y.__contains__(-1)

False

In [20]:
# Make the NDGCAHMO into a single array
# to categorical
y_train_categorical = keras.utils.to_categorical(
    y_train, num_classes=9
)
y_test_categorical = keras.utils.to_categorical(
    y_test, num_classes=9
)

In [23]:
y_train_categorical.shape
y_train_categorical[0]

array([0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [25]:
from keras import layers
from keras import models

In [33]:
def initialize_model():
  model = models.Sequential()

  model.add(layers.Conv2D(32, (3,3), input_shape=(256, 256, 3), activation='relu', padding='same'))
  model.add(layers.MaxPool2D(pool_size=(2,2)))
  model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same'))
  model.add(layers.MaxPool2D(pool_size=(2,2)))
  model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
  model.add(layers.MaxPool2D(pool_size=(3,3)))
  model.add(layers.Conv2D(256, (3,3), activation='relu', padding='same'))
  model.add(layers.MaxPool2D(pool_size=(3,3)))
  ### Flattening
  model.add(layers.Flatten())
  ### One fully connected
  model.add(layers.Dense(120, activation='relu'))
  model.add(layers.Dropout(rate=0.5))
  model.add(layers.Dense(60, activation='relu'))
  model.add(layers.Dropout(rate=0.5))
  model.add(layers.Dense(9, activation='sigmoid'))

  model.compile(loss='categorical_crossentropy', 
        optimizer='adam',
        metrics=['accuracy'])

  return model

In [34]:
model = initialize_model()
# model.summary()

c:\Users\Hamza\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
from keras.callbacks import EarlyStopping

In [35]:
model = initialize_model()
es = EarlyStopping(patience=20, restore_best_weights=True)

history = model.fit(X_train, y_train_categorical,
                    validation_split=0.3,
                    epochs=1,
                    batch_size=16, 
                    verbose=1,
                    callbacks=[es])

188/188 ━━━━━━━━━━━━━━━━━━━━ 209s 1s/step - accuracy: 0.3493 - loss: 1.8521 - val_accuracy: 0.4712 - val_loss: 1.6526


In [36]:
model.evaluate(X_test, y_test_categorical, verbose=0)

[1.6766080856323242, 0.4558342397212982]